In [2]:
import numpy as np

In [57]:
def pad_input_matrix(X, pad: int, padding_value: int = 0):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C) # m is omitted for now
    """
    X_pad = np.pad(X, pad_width=((0, 0), (pad, pad), (pad, pad)),
                   mode='constant', constant_values=(padding_value, padding_value))
    # X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)))

    return X_pad

In [ ]:
class ConvolutionalLayer:
    # number of output channel = filter counts
    def __init__(self, input_shape: (int, int, int), number_of_output_channel: int, filter_dimension: int,
                 stride: int, padding: int):
        self.input = None
        self.output = None
        self.number_of_output_channel = number_of_output_channel
        self.filter_dimension = filter_dimension
        self.stride = stride
        self.padding = padding

        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.input_depth = input_depth

        self.output_shape = (number_of_output_channel, input_height - filter_dimension + 1, input_width
                             - filter_dimension + 1)

        self.filters_shape = (number_of_output_channel, input_depth, filter_dimension, filter_dimension)
        self.filters = np.random.randn(*self.filters_shape)

        self.biases = np.random.randn(*self.output_shape)


    def convolution_single_step(input_slice, filter_mat):
        """
        Apply one filter defined by parameters weight on a single slice (a_slice_prev) of the output activation
        of the previous layer.

        Arguments:
        a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
        weight -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
        bias -- Bias parameters contained in a window - matrix of shape (1, 1, 1)

        Returns:
        Z -- a scalar value, the result of convolving the sliding window (weight, bias) on a slice x of the input data
        """

        #(≈ 3 lines of code)
        # Element-wise product between a_slice_prev and weight. Do not add the bias yet.
        s = np.multiply(a_slice_prev, weight)
        # Sum over all entries of the volume s.
        Z = np.sum(s)
        # Add bias to Z. Cast bias to a float() so that Z results in a scalar value.
        bias = np.squeeze(bias)
        Z = Z + bias

        return Z


    def conv_forward(self, A_prev, W, b):
        """
        Implements the forward propagation for a convolution function

        Arguments:
        A_prev -- output activations of the previous layer,
            numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
        W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
        b -- Biases, numpy array of shape (1, 1, 1, n_C)
        hparameters -- python dictionary containing "stride" and "pad"

        Returns:
        Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
        cache -- cache of values needed for the conv_backward() function
        """

        # Retrieve dimensions from A_prev's shape (≈1 line)
        (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

        # Retrieve dimensions from W's shape (≈1 line)
        (f, f, n_C_prev, n_C) = W.shape

        # Retrieve information from "hparameters" (≈2 lines)
        # stride = hparameters["stride"]
        # pad = hparameters["pad"]

        # Compute the dimensions of the CONV output volume using the formula given above.
        # Hint: use int() to apply the 'floor' operation. (≈2 lines)
        n_H = int((n_H_prev + 2*self.padding - f)/self.stride) + 1
        n_W = int((n_W_prev + 2*self.padding - f)/self.stride) + 1

        # Initialize the output volume Z with zeros. (≈1 line)
        Z = np.zeros((m, n_H, n_W, n_C))

        # Create A_prev_pad by padding A_prev
        A_prev_pad = zero_pad(A_prev, self.padding)

        for i in range(m):               # loop over the batch of training examples
            a_prev_pad = A_prev_pad[i]          # Select ith training example's padded activation
            for h in range(n_H):           # loop over vertical axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                vert_start = self.stride * h
                vert_end = vert_start  + f

                for w in range(n_W):       # loop over horizontal axis of the output volume
                    # Find the horizontal start and end of the current "slice" (≈2 lines)
                    horiz_start = self.stride * w
                    horiz_end = horiz_start + f

                    for c in range(n_C):   # loop over channels (= #filters) of the output volume

                        # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                        a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]

                        # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                        weights = W[:, :, :, c]
                        biases  = b[:, :, :, c]
                        Z[i, h, w, c] = self.convolution_single_step(a_slice_prev, weights, biases)

        # Save information in "cache" for the backprop
        cache = (A_prev, W, b, hparameters)

        return Z, cache
    


In [41]:
np.random.seed(1)
x = np.random.randn(2, 2)
s, _ = x.shape
print(x[0])
print(s)

[ 1.62434536 -0.61175641]
2


In [34]:
temp = np.floor(3.7)
print(temp)

3.0


In [7]:
a = [[1, 2], [3, 40]]
a = np.pad(a, pad_width=(1, 1)  mode='constant', constant_values=(0, 0))
print(a)

[[ 0  0  0  0]
 [ 0  1  2  0]
 [ 0  3 40  0]
 [ 0  0  0  0]]


In [100]:
def flatten_matrix(input):
    return input.flatten().reshape(-1, 1)

x = np.random.randn(3, 2, 2)
y = flatten_matrix(x)
print(x)
print(y)
print(y.shape)

[[[-0.91797604 -0.85183742]
  [ 0.09075857  1.16220247]]

 [[ 0.32178247  1.34131783]
  [ 0.14167518  1.73738467]]

 [[ 0.37576878  1.47825568]
  [-2.08016719 -0.45342757]]]
[[-0.91797604]
 [-0.85183742]
 [ 0.09075857]
 [ 1.16220247]
 [ 0.32178247]
 [ 1.34131783]
 [ 0.14167518]
 [ 1.73738467]
 [ 0.37576878]
 [ 1.47825568]
 [-2.08016719]
 [-0.45342757]]
(12, 1)


In [87]:

x = np.random.randint(low = 1, high = 8, size=(2, 2))
y = np.random.randint(low = 1, high = 8, size=(2, 2))
print(x)
print(y)
print(np.multiply(x, y))
print(np.sum(np.multiply(x, y)))

[[2 4]
 [5 1]]
[[2 4]
 [5 3]]
[[ 4 16]
 [25  3]]
48


In [101]:
kernel = np.random.randint(low=1, high=10, size=(3,3))
print(kernel)
kernel = np.flipud(np.fliplr(kernel))
print(kernel)

[[8 9 9]
 [9 1 8]
 [3 1 8]]
[[8 1 3]
 [8 1 9]
 [9 9 8]]


In [103]:
import numpy as np


def file_read(file_path: str):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    lines = [line.split() for line in lines]
    lines = [[int(x) if idx != 0 else x for idx, x in enumerate(line)] for line in lines]

    '''
    *********************************** INPUT FORMAT *********************************************
    Conv: the number of output channels, filter dimension, stride, padding.
    ReLU: Activation layer.
    Pool: filter dimension, stride.
    FC: output dimension.
    Flattening layer: it will convert a (series of) convolutional filter maps to a column vector.
    Softmax: it will convert final layer projections to normalized probabilities.
    '''

    return lines


def pad_input_2d_mat(X, pad: int, padding_value: int = 0):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C) # m is omitted for now
    """
    X_pad = np.pad(X, pad_width=((pad, pad), (pad, pad)),
                   mode='constant', constant_values=(padding_value, padding_value))
    # X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)))

    return X_pad


def cross_correlation2d(input_mat, filter_mat, stride, padding, result_shape):
    filter_dimension, _ = filter_mat.shape
    _, result_height, result_width = result_shape

    result = np.zeros((result_height, result_width), dtype=float)

    if padding != 0:
        pad_input = pad_input_2d_mat(input_mat, pad=padding)
    else:
        pad_input = input_mat

    for h in range(result_height):  # loop over vertical axis of the output volume
        # Find the vertical start and end of the current "slice" (≈2 lines)
        vertical_start = stride * h
        vertical_end = vertical_start + filter_dimension

        for w in range(result_width):  # loop over horizontal axis of the output volume
            # Find the horizontal start and end of the current "slice" (≈2 lines)
            horizontal_start = stride * w
            horizontal_end = horizontal_start + filter_dimension

            input_slice = pad_input[vertical_start:vertical_end, horizontal_start:horizontal_end]
            result[h, w] = np.sum(np.multiply(input_slice, filter_mat))
            # print(result[h, w])

    return result


def convolution2d(input_mat, filter_mat, stride, padding, result_shape):
    # Flip the filter
    rotated_filter = np.flipud(np.fliplr(filter_mat))

    return cross_correlation2d(input_mat, rotated_filter, stride, padding, result_shape)


def flatten_matrix(input_data):
    return input_data.flatten().reshape(-1, 1)

In [7]:
import numpy as np
np.random.seed(1)
x = np.random.randn(100, 5, 4, 4)
print(x.shape[1:])


(5, 4, 4)


# CIFAR dataset

## load daatset

In [67]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [68]:
print(len(x_train))
print(y_train.shape)
print(y_train.shape)

60000
(60000,)
(60000,)


In [64]:
from keras.utils import np_utils
import numpy as np

def preprocess_mnist_data(x, y):
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float64") / 255
    # y = np_utils.to_categorical(y)
    y = y.reshape(-1, 1)

    return x, y

In [65]:
x_train, y_train = preprocess_mnist_data(x_train, y_train)
x_test, y_test = preprocess_mnist_data(x_test, y_test)


ValueError: cannot reshape array of size 153600000 into shape (50000,1,28,28)

In [43]:
print(x_train.shape)
print(y_train.shape)



(60000, 1, 28, 28)
(60000, 1)


TypeError: set_printoptions() got multiple values for argument 'precision'

## CIFAR10 dataset load

In [69]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [71]:
print(y_train.shape)

(50000, 1)


In [63]:
new_x_train = np.transpose(x_train, (0, 3, 1, 2))
new_x_test = np.transpose(x_test, (0, 3, 1, 2))

print(new_x_train.shape)
print(new_x_test.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [66]:
new_x_train[0]

array([[[ 59,  43,  50, ..., 158, 152, 148],
        [ 16,   0,  18, ..., 123, 119, 122],
        [ 25,  16,  49, ..., 118, 120, 109],
        ...,
        [208, 201, 198, ..., 160,  56,  53],
        [180, 173, 186, ..., 184,  97,  83],
        [177, 168, 179, ..., 216, 151, 123]],

       [[ 62,  46,  48, ..., 132, 125, 124],
        [ 20,   0,   8, ...,  88,  83,  87],
        [ 24,   7,  27, ...,  84,  84,  73],
        ...,
        [170, 153, 161, ..., 133,  31,  34],
        [139, 123, 144, ..., 148,  62,  53],
        [144, 129, 142, ..., 184, 118,  92]],

       [[ 63,  45,  43, ..., 108, 102, 103],
        [ 20,   0,   0, ...,  55,  50,  57],
        [ 21,   0,   8, ...,  50,  50,  42],
        ...,
        [ 96,  34,  26, ...,  70,   7,  20],
        [ 96,  42,  30, ...,  94,  34,  34],
        [116,  94,  87, ..., 140,  84,  72]]], dtype=uint8)

In [47]:
f_size = (1,2,3,4)
print(*f_size)
ara = np.random.randn(*f_size)
ara1 = np.random.sample(f_size)

# print(ara)
# print(ara1)
s = f_size[1:]
print(s)


1 2 3 4


AttributeError: 'tuple' object has no attribute 'shape'

In [12]:
from keras.utils import np_utils
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10

def preprocess_mnist_data(x, y):
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float64") / 255
    y = y.reshape(-1, 1)
    y = np_utils.to_categorical(y) # confusion

    return x, y


def preprocess_cifar10_data(x, y): # dimension (60k, 32, 32, 3)
    x = np.transpose(x, (0, 3, 1, 2)) # changed dimension (60k, 3, 32, 32)
    x = x.astype("float64") / 255
    y = np_utils.to_categorical(y) # confusion

    return x, y


def mnist_dataset_load_and_preprocess():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, y_train = preprocess_mnist_data(x_train, y_train)
    x_test, y_test = preprocess_mnist_data(x_test, y_test)

    test_len = int(len(x_test)/2)

    x_test_new = x_test[0:test_len]
    y_test_new = y_test[0:test_len]

    x_valid = x_test[test_len:]
    y_valid = y_test[test_len:]

    return x_train, y_train, x_valid, y_valid, x_test_new, y_test_new


def cifar10_dataset_load_and_preprocess():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()  # dimension (60k, 32, 32, 3)
    x_train, y_train = preprocess_cifar10_data(x_train, y_train)
    x_test, y_test = preprocess_cifar10_data(x_test, y_test)

    test_len = int(len(x_test)/2)

    x_test_new = x_test[0:test_len]
    y_test_new = y_test[0:test_len]

    x_valid = x_test[test_len:]
    y_valid = y_test[test_len:]

    return x_train, y_train, x_valid, y_valid, x_test_new, y_test_new

In [13]:
x_train, y_train, x_valid, y_valid, x_test, y_test = mnist_dataset_load_and_preprocess()

In [77]:
x_train, y_train, x_test, y_test = cifar10_dataset_load_and_preprocess()

In [17]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)


(5000, 1, 28, 28)
(5000, 10)
(60000, 1, 28, 28)
(60000, 10)


In [80]:
x_test[0].shape

(3, 32, 32)

In [81]:
kernel = np.array([[1, 2], [-1, 0]])
print(kernel)
kernel = np.flipud(np.fliplr(kernel))
print(kernel)

[[ 1  2]
 [-1  0]]
[[ 0 -1]
 [ 2  1]]


### signal convolution check

In [83]:
def cross_correlation2d(input_mat, filter_mat, stride, padding, result_shape):
    filter_dimension = filter_mat.shape[0]
    result_height, result_width = result_shape

    result = np.zeros((result_height, result_width), dtype='float64')

    if padding > 0:
        pad_input = pad_input_2d_mat(input_mat, pad=padding)
    else:
        pad_input = input_mat

    for h in range(result_height):  # loop over vertical axis of the output volume
        # Find the vertical start and end of the current "slice" (≈2 lines)
        vertical_start = stride * h
        vertical_end = vertical_start + filter_dimension

        for w in range(result_width):  # loop over horizontal axis of the output volume
            # Find the horizontal start and end of the current "slice" (≈2 lines)
            horizontal_start = stride * w
            horizontal_end = horizontal_start + filter_dimension

            input_slice = pad_input[vertical_start:vertical_end, horizontal_start:horizontal_end]
            result[h, w] = np.sum(np.multiply(input_slice, filter_mat))
            # print(result[h, w])

    return result


def convolution2d(input_mat, filter_mat, stride, padding, result_shape):
    # Flip the filter
    rotated_filter = np.flipud(np.fliplr(filter_mat))

    return cross_correlation2d(input_mat, rotated_filter, stride, padding, result_shape)


In [90]:
from scipy.signal import convolve2d, correlate2d
w = np.array([[1, 6, 2], [5, 3, 1], [7, 0, 4]])
print(w)
kernel = np.array([[1, 2], [-1, 0]])
print(kernel)
out1 = correlate2d(w, kernel, mode='valid')
print(out1)
out2 = cross_correlation2d(w, kernel, 2, 0, (2,2))
print(out2)

[[1 6 2]
 [5 3 1]
 [7 0 4]]
[[ 1  2]
 [-1  0]]
[[8 7]
 [4 5]]
[[8.]]


In [95]:
x = np.random.randint(low = 1, high = 8, size=(2, 3, 3))
y = np.random.randint(low = 1, high = 8, size=(2, 3, 3))
print(np.dot(x, y))
print(np.matmul(x, y))

[[[[37 34 24]
   [47 17 24]]

  [[52 54 68]
   [78 48 56]]

  [[67 62 47]
   [79 28 46]]]


 [[[67 63 53]
   [78 30 50]]

  [[43 46 60]
   [89 59 48]]

  [[59 57 51]
   [96 48 46]]]]
[[[37 34 24]
  [52 54 68]
  [67 62 47]]

 [[78 30 50]
  [89 59 48]
  [96 48 46]]]


In [23]:
import numpy as np
x = np.random.randint(low = 1, high = 8, size=(10, 1))
print(x)
print(len(x))

a = [1,2,3]

for i in a:
    print(i)
print('hello')
for i in reversed(a):
    print(i)
print(a)

[[5]
 [6]
 [4]
 [1]
 [2]
 [3]
 [1]
 [6]
 [7]
 [5]]
10
1
2
3
hello
3
2
1
[1, 2, 3]


In [29]:
arr = np.zeros((10,1))
arr[1] = [456]
print(arr)

[[  0.]
 [456.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]]


In [3]:
class test:
    def __init__(self):
        self.a = 6
        self.g='eefef'

t = test()
print(t.g)

eefef


In [21]:
def pad_input_2d_mat(X, pad: int, padding_value: int = 0):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C) # m is omitted for now
    """
    X_pad = np.pad(X, pad_width=((pad, pad), (pad, pad)),
                   mode='constant', constant_values=(padding_value, padding_value))
    # X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)))

    return X_pad

In [15]:
def relu_derivative(x):
    # x[x < 0] = 0.01 * x  # leaky ReLU
    # x[x >= 0] = 1
    x = np.where(x> 0, x, x * 0.01)
    # new_window = (specific_window == np.max(specific_window))
    return x


In [22]:
import numpy as np
x = np.random.randint(low=0, high=12, size=(5,10))
s = x.shape + (1, )

print(s)
x = np.reshape(x, s)
print(x)

(5, 10, 1)
[[[ 5]
  [ 4]
  [ 4]
  [ 8]
  [ 7]
  [ 8]
  [11]
  [ 9]
  [ 7]
  [ 0]]

 [[ 6]
  [11]
  [ 0]
  [ 8]
  [ 2]
  [ 2]
  [ 2]
  [ 0]
  [ 9]
  [ 7]]

 [[ 2]
  [ 0]
  [ 3]
  [ 9]
  [ 3]
  [ 2]
  [ 7]
  [11]
  [11]
  [ 8]]

 [[ 1]
  [ 4]
  [ 7]
  [ 6]
  [ 1]
  [ 5]
  [ 5]
  [ 3]
  [ 0]
  [ 6]]

 [[ 5]
  [ 6]
  [ 4]
  [ 2]
  [ 0]
  [ 7]
  [11]
  [ 8]
  [ 1]
  [ 8]]]
